In [10]:
import numpy as np
import pandas as pd

In [11]:
# import dataset
df = pd.read_csv("NSDUH_2022.txt", sep='\t', header=0)

# output dataframe
df

C:\Users\seren\AppData\Local\Temp\ipykernel_27532\483449733.py:2: DtypeWarning: Columns (2604) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("NSDUH_2022.txt", sep='\t', header=0)


,QUESTID2,FILEDATE,ANALWT2_C,VESTR_C,VEREP,PDEN10,COUTYP4,MAIIN102,AIIND102,AGE3,...,COSUTELE2,COSUAPTDL2,COSURXDL2,COSUSVHLT2,COHCTELE2,COHCAPTDL2,COHCRXDL2,COHCSVHLT2,LANGVER,GQTYPE2
0,10000330,10/19/2023,298.004021,40030,2,1,1,2,2,6,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2,NaN
1,10000425,10/19/2023,2292.377696,40002,1,2,2,2,2,9,...,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,1,NaN
2,10002771,10/19/2023,216.604339,40041,2,1,1,2,2,6,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1,NaN
3,10005250,10/19/2023,13828.218374,40012,2,2,1,2,2,7,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1,NaN
4,10005432,10/19/2023,6282.612060,40037,1,2,2,2,2,10,...,3.0,3.0,3.0,3.0,2.0,1.0,2.0,2.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50505,46071114,10/19/2023,11661.588421,40013,1,2,2,2,2,9,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1,NaN
50506,46071268,10/19/2023,1876.215399,40034,2,2,2,2,2,9,...,3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1,NaN
50507,46071830,10/19/2023,3216.773493,40031,1,1,1,2,2,8,...,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,1,NaN
50508,46072042,10/19/2023,498.719872,40003,1,2,2,2,2,2,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1,NaN


In [12]:
# retrieve all columns
cols = df.columns.tolist()

# demographic data indexes
dem_index_start = cols.index("AGE3")
dem_index_end = cols.index("II2WRKST18")

# pain reliever data indexes
pr_index_start = cols.index("OXCNANYYR")
pr_index_end = cols.index("FENTNMYR")

# mental health data indexes
mh_index_start = cols.index("DSTNRV30")
mh_index_end = cols.index("AALTMDE")

# covid data indexes
cov_index_start = cols.index("CONEGMH")
cov_index_end = cols.index("COHCSVHLT")

# create new df with select columns
df2 = pd.merge(df.loc[:, "QUESTID2"], # respondent identification
               df.iloc[:, np.r_[0, dem_index_start:dem_index_end]], # demographics
               on="QUESTID2")
df2 = pd.merge(df2,
               df.iloc[:, np.r_[0, pr_index_start:pr_index_end]], # pain reliever substance use
               on="QUESTID2")
df2 = pd.merge(df2,
               df.iloc[:, np.r_[0, mh_index_start:mh_index_end]], # adult mental health 
               on="QUESTID2")
df2 = pd.merge(df2,
               df.iloc[:, np.r_[0, cov_index_start:cov_index_end]], # covid mental health 
               on="QUESTID2")

# output dataframe
df2

,QUESTID2,AGE3,NOMARR2,SERVICE,MILSTAT,ACTDEVER,ACTD2001,ACTD9001,ACTD7590,ACTDVIET,...,COMHAPTDL,COMHRXDL,COMHSVHLT,COSUTELE,COSUAPTDL,COSURXDL,COSUSVHLT,COHCTELE,COHCAPTDL,COHCRXDL
0,10000330,6,1,2,99,99,99,99,99,99,...,1,1,2,1,1,1,2,1,1,1
1,10000425,9,99,2,99,99,99,99,99,99,...,2,2,2,3,3,3,3,2,2,2
2,10002771,6,1,2,99,99,99,99,99,99,...,3,3,3,3,3,3,3,3,3,3
3,10005250,7,1,2,99,99,99,99,99,99,...,2,2,2,2,2,2,2,2,2,2
4,10005432,10,1,2,99,99,99,99,99,99,...,1,2,2,3,3,3,3,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50505,46071114,9,1,2,99,99,99,99,99,99,...,3,3,3,3,3,3,3,3,3,3
50506,46071268,9,99,2,99,99,99,99,99,99,...,1,1,2,3,3,3,3,1,1,1
50507,46071830,8,1,2,99,99,99,99,99,99,...,1,1,1,3,1,3,3,3,3,3
50508,46072042,2,99,99,99,99,99,99,99,99,...,2,2,2,2,2,2,2,2,2,2


In [13]:
# check for NA values
df2.isna().sum()

QUESTID2     0
AGE3         0
NOMARR2      0
SERVICE      0
MILSTAT      0
            ..
COSURXDL     0
COSUSVHLT    0
COHCTELE     0
COHCAPTDL    0
COHCRXDL     0
Length: 370, dtype: int64